In [1]:
from marvin.tools.query import Query
from marvin.tools.maps import Maps
import numpy as np

INFO: No release version set. Setting default to DR15


/opt/python/3.7.3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/python/3.7.3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/python/3.7.3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/opt/python/3.7.3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
tot = [] #All galaxies
agns = [] #AGNs

In [3]:
def get_masked(maps, emline):
    """Function to get masked data arrays without bad snr spaxels

    Parameters:
    maps - Marvin Maps object of the galaxy
    emline - String of desired quantity (e.g. 'emline_gflux_ha_6564')

    Returns:
    Data array with masked values if the spaxel has an snr < 5
    """    
    data_array = maps[emline]
    data_array_masked = data_array.masked

    #Masks all spaxels that don't reach the cutoff SNR
    data_array_masked.mask |= maps.bin_snr.value < 5
    
    return data_array_masked

In [4]:
def get_trues(arr):
    """Function to count the number of True bools in a numpy array

    Parameters:
    arr - numpy array containing boolean values

    Returns:
    Number of True values in arr
    """
    counter = 0
    for elem in arr.compressed():
        if elem:
            counter += 1
    return counter

In [5]:
def get_ew_val(arr, arr_bools):
    """Function to calculate the mean of the top 20% of the ews for
    a specific subset of the spaxels

    Parameters:
    arr - Data array of ews
    arr_bools - Corresponding array containing bools which specify whether
                each spaxel is a part of the subset of ews being calculated

    Returns:
    Mean of the top 20% of the subset of ews
    """
    ewhas_list = []
    for i in range(len(arr)):
        for j in range(len(arr)):
            if arr_bools[i][j] and not np.ma.is_masked(arr[i][j]):
                ewhas_list.append(arr[i][j])
                
    ewhas = np.array(ewhas_list)
    
    num_20 = round(len(ewhas)*0.2)
    ewhas_20 = ewhas[np.argsort(ewhas)[-num_20:]]
    
    return np.mean(ewhas_20)

In [6]:
def combine_bool_arr(arr_1, arr_2):
    """Combines data arrays containing booleans according to "or" logic

    Parameters:
    arr_1 - First data array containing bools
    arr_2 - Second data array containing bools

    Returns:
    Returns new array which contains True in each position that either 
    arr_1 or arr_2 contains True
    """      
    arr_1_copy = arr_1.copy()
    
    for i in range(len(arr_2)):
        for j in range(len(arr_2)):
            if arr_2[i][j]:
                arr_1_copy.mask[i][j] = False
                arr_1_copy[i][j] = True

    return arr_1_copy

In [7]:
def classify_gal(gal_id):
    """Uses fraction of spaxels considered AGN, comp., LINER, etc. as well as
    the mean of the top 20% of the ews for these spaxels to determine whether
    a galaxy hosts an AGN (Wylezalek et al. 2017)

    Parameters:
    gal_id - MaNGA id of the galaxy

    Returns:
    Function has no return value, however, appends galaxies that
    fulfill the criteria to the list of AGNs
    """    
    try:
        maps = Maps(mangaid=gal_id) 
    except:
        return None
    
    if 28 in maps.quality_flag.bits or 29 in maps.quality_flag.bits or 30 in maps.quality_flag.bits:
        print('Critical Flag!\n')
        return None
    
    ha = get_masked(maps, 'emline_gflux_ha_6564')
    nii = get_masked(maps, 'emline_gflux_nii_6585')
    hb = get_masked(maps, 'emline_gflux_hb_4862')
    oiii = oiii = get_masked(maps, 'emline_gflux_oiii_5008')
    
    sii_6718 = get_masked(maps, 'emline_gflux_sii_6718')
    sii_6732 = get_masked(maps, 'emline_gflux_sii_6732')
    sii = sii_6718 + sii_6732
    
    nii_count = nii.count()
    sii_count = sii.count()
    
    log_nii_ha = np.log10(nii/ha)
    log_oiii_hb = np.log10(oiii/hb)
    log_sii_ha = np.log10((sii_6718 + sii_6732)/ha)
    
    nii_agn_bools = (0.61/(log_nii_ha - 0.47) + 1.19 < log_oiii_hb)
    nii_agn = get_trues(nii_agn_bools)
    
    nii_comp_bools = (0.61/(log_nii_ha - 0.47) + 1.19 > log_oiii_hb) & (0.61/(log_nii_ha - 0.05) + 1.3 < log_oiii_hb)
    nii_comp = get_trues(nii_comp_bools)
    
    sii_agn_bools = ((0.72/(log_sii_ha - 0.32) + 1.30 < log_oiii_hb) & (1.89*log_sii_ha + 0.76 < log_oiii_hb))
    sii_agn = get_trues(sii_agn_bools)
    
    sii_liner_bools = ((0.72/(log_sii_ha - 0.32) + 1.30 < log_oiii_hb) & (1.89*log_sii_ha + 0.76 > log_oiii_hb))
    sii_liner = get_trues(sii_liner_bools)
    
    f_A_N = (80*nii_agn + 20*nii_comp)/nii_count
    f_A_S = 100*sii_agn/sii_count
    f_L_S = 100*sii_liner/sii_count
    f_AL_S = f_A_N + f_L_S
    
    ewha = get_masked(maps, 'emline_sew_ha_6564')
    
    nii_agn_comp_bools = combine_bool_arr(nii_agn_bools, nii_comp_bools)
    
    ewha_A_N = get_ew_val(ewha, nii_agn_comp_bools)
    ewha_A_S = get_ew_val(ewha, sii_agn_bools)
    ewha_L_S = get_ew_val(ewha, sii_liner_bools)
    ewha_AL_S = (ewha_A_S + ewha_L_S)/2
    
    if (f_A_N > 15 and ewha_A_N > 5) or (f_AL_S > 15 and ewha_AL_S > 5):
        agns.append(gal_id)

In [8]:
q = Query(limit=10000)
r = q.run()

for i in range(r.totalcount):
    tot.append(r.results[i].mangaid)
    classify_gal(r.results[i].mangaid)
    print(i)

/home/student10/.local/lib/python3.7/site-packages/marvin/tools/query.py:241: MarvinUserWarning: No local database found. Cannot perform queries.
  warnings.warn('No local database found. Cannot perform queries.', MarvinUserWarning)
/opt/python/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log10
/opt/python/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in log10


0
1
2
3
4
5
6
7
8


/opt/python/3.7.3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/python/3.7.3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


9
10
11
12
13
14
15
16
17
Critical Flag!

18
19
20
21
22
23
Critical Flag!

24
25
26
27
28
29
30


/home/student10/.local/lib/python3.7/site-packages/marvin/utils/general/general.py:356: MarvinUserWarning: more than one plate-ifu found for mangaid=1-113375. Using the one with the highest SN2.
  MarvinUserWarning)


...

4834
4835


/home/student10/.local/lib/python3.7/site-packages/marvin/utils/general/general.py:356: MarvinUserWarning: more than one plate-ifu found for mangaid=52-16. Using the one with the highest SN2.
  MarvinUserWarning)


4836
4837


/home/student10/.local/lib/python3.7/site-packages/marvin/utils/general/general.py:356: MarvinUserWarning: more than one plate-ifu found for mangaid=52-17. Using the one with the highest SN2.
  MarvinUserWarning)


4838
4839
4840
4841
4842
4843
4844
4845
4846
4847
4848
4849
4850
4851
4852
4853
4854
4855
4856


In [9]:
print(len(tot))

4857


In [10]:
print(tot)

['1-109056', '1-109073', '1-109081', '1-109112', '1-109152', '1-109167', '1-109234', '1-109244', '1-109250', '1-109270', '1-109284', '1-109308', '1-109356', '1-109378', '1-109392', '1-109394', '1-109419', '1-109428', '1-109493', '1-109521', '1-113208', '1-113219', '1-113242', '1-113265', '1-113273', '1-113292', '1-113315', '1-113322', '1-113343', '1-113346', '1-113368', '1-113375', '1-113375', '1-113379', '1-113379', '1-113403', '1-113404', '1-113405', '1-113418', '1-113468', '1-113469', '1-113469', '1-113476', '1-113518', '1-113520', '1-113521', '1-113525', '1-113525', '1-113525', '1-113539', '1-113540', '1-113551', '1-113567', '1-113567', '1-113585', '1-113587', '1-113647', '1-113651', '1-113654', '1-113663', '1-113668', '1-113672', '1-113675', '1-113698', '1-113700', '1-113712', '1-114073', '1-114082', '1-114121', '1-114128', '1-114129', '1-114145', '1-114171', '1-114303', '1-114306', '1-114325', '1-114334', '1-114454', '1-114465', '1-114500', '1-114502', '1-114532', '1-114928', '1-

In [11]:
print(len(agns))

1393


In [12]:
print(agns)

['1-109056', '1-109244', '1-109250', '1-109394', '1-113265', '1-113292', '1-113368', '1-113379', '1-113379', '1-113403', '1-113405', '1-113418', '1-113539', '1-113587', '1-113651', '1-113663', '1-113672', '1-113700', '1-113712', '1-114129', '1-114145', '1-114303', '1-114325', '1-114334', '1-114454', '1-114955', '1-114956', '1-114980', '1-121075', '1-121151', '1-121209', '1-121523', '1-121532', '1-121717', '1-121773', '1-121973', '1-121994', '1-122054', '1-122057', '1-122059', '1-122067', '1-122094', '1-122119', '1-122284', '1-122304', '1-122361', '1-122412', '1-133941', '1-134239', '1-134293', '1-134597', '1-134620', '1-134695', '1-134716', '1-134737', '1-134954', '1-134994', '1-135030', '1-135044', '1-135152', '1-135239', '1-135285', '1-135371', '1-135383', '1-135491', '1-135500', '1-135516', '1-135524', '1-135530', '1-135534', '1-135536', '1-135545', '1-135568', '1-135608', '1-135625', '1-135626', '1-135641', '1-135810', '1-136125', '1-136286', '1-137528', '1-137883', '1-137912', '1-

In [14]:
with open('AGN Wylezalek MaNGAids.txt', 'w') as f:
    for gal_id in agns:
        f.write("%s\n" % gal_id)